In [33]:
class Solution:
    def isMatch(self, s: str, p: str) -> bool:
        # goal here is to see if the string s matches the pattern p
        # where p can have '.' as a wildcard for any single character
        # and '*' as a wildcard for matching 0 or more of the preceding character

        # lets do this recursively. Each call will consume 1-2 characters from s 
        # and 0 or more characters from p. This way we only have to worry about 
        # a tiny part of the pattern and string at a time

        def hasWildcard(p: str) -> bool:
            return len(p) >= 2 and p[1] == '*'

        ns = len(s)
        np = len(p)
        #print(f"s: {s}, p: {p}")

        # if the string is empty, the pattern must be empty or have wildcards
        if ns == 0:
            if hasWildcard(p):
                return self.isMatch(s, p[2:])
            return np == 0
        
        # if the pattern is empty, the string must also be empty
        if np == 0:
            return ns == 0
        
        if hasWildcard(p):
            # if the next character in the pattern is a wildcard
            # consume characters from the string until we can't anymore
            # match zero times
            return self.isMatch(s, p[2:]) or (s[0] == p[0] or p[0] == '.') and self.isMatch(s[1:], p)
        else:
            # if the next character in the pattern is not a wildcard
            # check for a match and consume the character
            if s[0] == p[0] or p[0] == '.':
                return self.isMatch(s[1:], p[1:])
        
        return False

In [40]:
class Solution:
    def isMatch(self, s: str, p: str) -> bool:
        memo = {}  # Cache our results
        
        def hasWildcard(p: str) -> bool:
            return len(p) >= 2 and p[1] == '*'
        
        def match(s: str, p: str) -> bool:
            # If we've seen this combination before, return cached result
            if (s, p) in memo:
                return memo[(s, p)]
            
            ns = len(s)
            np = len(p)

            # if the string is empty, the pattern must be empty or have wildcards
            if ns == 0:
                if hasWildcard(p):
                    result = match(s, p[2:])
                else:
                    result = np == 0
                memo[(s, p)] = result
                return result
            
            # if the pattern is empty, the string must also be empty
            if np == 0:
                result = ns == 0
                memo[(s, p)] = result
                return result
            
            if hasWildcard(p):
                # if the next character in the pattern is a wildcard
                # try both possibilities: use it or don't use it
                result = match(s, p[2:]) or ((s[0] == p[0] or p[0] == '.') and match(s[1:], p))
            else:
                # if the next character in the pattern is not a wildcard
                # must match and consume one character
                if s[0] == p[0] or p[0] == '.':
                    result = match(s[1:], p[1:])
                else:
                    result = False
            
            memo[(s, p)] = result
            return result
        
        return match(s, p)

In [41]:
s = "bbc"
p = "b*.*."
result = Solution().isMatch(s, p)
print(f"result: {result}, expected: True")

s = "aa"
p = "a"
result = Solution().isMatch(s, p)
print(f"result: {result}, expected: False")

s = "aa"
p = "a*"
result = Solution().isMatch(s, p)
print(f"result: {result}, expected: True")

s = "abacacccbbbcbcbb"
p = ".*.*.*ab*.*ab.*c*"
result = Solution().isMatch(s, p)
print(f"result: {result}, expected: False")

s = "aaaaaaaaaaaaaaaaaaab"
p = "a*a*a*a*a*a*a*a*a*a*"
result = Solution().isMatch(s, p)
print(f"result: {result}, expected: False")

result: True, expected: True
result: False, expected: False
result: True, expected: True
result: False, expected: False
result: False, expected: False
